In [1]:
import numpy as np
import pandas as pd
import pymysql

In [9]:
#User can insert their comments and ratings into our database, the avgrating in Player_Bio will also update
def user_comments(PlayerName, UserName, ratings, Comments):
    #insert new entry in User_ratings
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB")
    #Get correct ratingID
    my_cursor = conn.cursor()
    my_cursor.execute("SELECT MAX(ratingID) FROM User_ratings")
    my_result = my_cursor.fetchall()
    new_ID = int(my_result[0][0]) + 1
    
    my_cursor = conn.cursor()
    sqlFormula = "INSERT INTO User_ratings (ratings, PlayerName, UserName, UserComments, ratingID) VALUES (%s, %s, %s, %s, %s)"
    records = (ratings, PlayerName, UserName, Comments, new_ID)
    my_cursor.execute(sqlFormula, records)
    conn.commit()
    
    #update in Player_Bio
    myplayer = PlayerName
    formula_1 = "SELECT Avg(ratings) as AvgRating FROM User_ratings WHERE PlayerName = %s"
    my_cursor.execute(formula_1, myplayer)
    my_result = my_cursor.fetchall()
    new_avgRatings = float(my_result[0][0])
    my_cursor = conn.cursor()
    formula_2 = "UPDATE Player_Bio SET AvgRating = %s WHERE PlayerName = %s"
    my_cursor.execute(formula_2, (new_avgRatings, myplayer))
    conn.commit()
    conn.close()
    print("Successfully commented, your commentID is %d." % new_ID)

#query Player_Bio table by PlayerName
def return_playerBio(PlayerName):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB")
    my_cursor = conn.cursor()
    formula_1 = "SELECT * FROM Player_Bio WHERE PlayerName = '%s'"
    sqlformula = formula_1 % PlayerName
    df = pd.read_sql(sqlformula, conn)
    print(df.to_string())
    conn.close()

#delete user rating and comments, then update average rating
def delete_ratings(deleteID):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB")
    
    #get playerName before deleting
    my_cursor = conn.cursor()
    formula_1 = "SELECT PlayerName FROM User_ratings WHERE ratingID = %s"
    my_cursor.execute(formula_1, deleteID)
    my_result = my_cursor.fetchall()
    Player_name = my_result[0][0]
    
    #delete record
    my_cursor = conn.cursor()
    sqlformula = "DELETE FROM User_ratings WHERE ratingID = %s"
    my_cursor.execute(sqlformula, deleteID)
    
    #update avgrating of the player in Player_Bio
    myplayer = Player_name
    formula_1 = "SELECT Avg(ratings) as AvgRating FROM User_ratings WHERE PlayerName = %s"
    my_cursor.execute(formula_1, myplayer)
    my_result = my_cursor.fetchall()
    new_avgRatings = 0
    if my_result[0][0]:
        new_avgRatings = float(my_result[0][0])
    my_cursor = conn.cursor()
    formula_2 = "UPDATE Player_Bio SET AvgRating = %s WHERE PlayerName = %s"
    my_cursor.execute(formula_2, (new_avgRatings, myplayer))
    conn.commit()
    print("Successfully deleted, the average rating of %s has been adjusted." % myplayer)
    conn.close()
    
    

In [53]:
# query player stats by playerName and Month
def query_stats_by_month(PlayerName, Month):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB")
    
    Months = {}
    Months["Jan"] = "01"
    Months["Feb"] = "02"
    Months["Mar"] = "03"
    Months["Apr"] = "04"
    Months["May"] = "05"
    Months["Jun"] = "06"
    Months["Jul"] = "07"
    Months["Aug"] = "08"
    Months["Sep"] = "09"
    Months["Oct"] = "10"
    Months["Nov"] = "11"
    Months["Dec"] = "12"

    my_cursor = conn.cursor()
    my_month = Months[Month]
    formula_1 = "SELECT PlayerName, Avg(Points) as Points, Avg(Rebounds) as Rebounds, Avg(Assists) as Assists, Avg(Steals) as Steals, count(*) as Game_Played FROM Game_Stats WHERE PlayerName = '%s' and CAST(Date AS CHAR) LIKE '____%s__'"
    sqlformula = formula_1 % (PlayerName, my_month)
    df = pd.read_sql(sqlformula, conn)
    print(df.to_string())
    conn.close()

def query_stats_by_opponent(PlayerName, Opponent):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB")

    my_cursor = conn.cursor()
    formula_1 = "SELECT PlayerName, OpponentTeam, Avg(Points) as Points, Avg(Rebounds) as Rebounds, Avg(Assists) as Assists, Avg(Steals) as Steals, count(*) as Game_Played FROM Game_Stats WHERE PlayerName = '%s' and OpponentTeam = '%s'"
    sqlformula = formula_1 % (PlayerName, Opponent)
    df = pd.read_sql(sqlformula, conn)
    print(df.to_string())
    conn.close()
    
def query_all_results_groupby_opponent(PlayerName):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB")

    my_cursor = conn.cursor()
    formula_1 = "SELECT PlayerName, OpponentTeam, Avg(Points) as Points, Avg(Rebounds) as Rebounds, Avg(Assists) as Assists, Avg(Steals) as Steals, count(*) as Game_Played FROM Game_Stats WHERE PlayerName = '%s' Group By OpponentTeam"
    sqlformula = formula_1 % (PlayerName)
    df = pd.read_sql(sqlformula, conn)
    print(df.to_string())
    conn.close()


# Who is the best clutch free throw shooter in team?

In [55]:
# More complicated query functions
# Find best free throw shooter, who has played more than 10 min in clutch moments
# for selected team：
def find_best_clutch_FT_Shooter(TeamName):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB")
    my_cursor = conn.cursor()

    my_cursor.execute('''Create view Jointed_Table as 
    select * from Player_Clutch_Stats natural join (select PlayerName, TeamName from Player_Bio) as short_bio;''')

    my_cursor.execute('''Create view BestFT_Shooters as
    Select PlayerName, Free_Throw_P, J.TeamName, J.Minutes_Played from Jointed_Table J, 
    (Select max(Free_Throw_P) as maxFT, TeamName from Jointed_Table where Minutes_Played > 10 group by TeamName) as bestFT
    where J.Free_Throw_P = bestFT.maxFT and J.TeamName = bestFT.TeamName and J.Minutes_Played > 10;''')

    formula_1 = "select PlayerName, Free_Throw_P, TeamName, Minutes_Played from BestFT_Shooters where TeamName = '%s';"
    sqlformula = formula_1 % TeamName
    df = pd.read_sql(sqlformula, conn)
    print(df.to_string())
    my_cursor.execute("Drop view Jointed_Table;")
    my_cursor.execute("Drop view BestFT_Shooters;")
    conn.close()

# Which team likes 3 pointer most?

In [123]:
def find_team_3PTA_ranking(TeamName):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "12345678",
    database = "NBA_DB")
    
    nbaTeams = {}
    nbaTeams['Atlanta Hawks'] = 'ATL'
    nbaTeams['Brooklyn Nets'] = 'BKN'
    nbaTeams['Boston Celtics'] = 'BOS'
    nbaTeams['Charlotte Hornets'] = 'CHA'
    nbaTeams['Chicago Bulls'] = 'CHI'
    nbaTeams['Cleveland Cavaliers'] = 'CLE'
    nbaTeams['Dallas Mavericks'] = 'DAL'
    nbaTeams['Denver Nuggets'] = 'DEN'
    nbaTeams['Detroit Pistons'] = 'DET'
    nbaTeams['Golden State Warriors'] = 'GSW'
    nbaTeams['Houston Rockets'] = 'HOU'
    nbaTeams['Indiana Pacers'] = 'IND'
    nbaTeams['LA Clippers'] = 'LAC'
    nbaTeams['Los Angeles Lakers'] = 'LAL'
    nbaTeams['Memphis Grizzlies'] = 'MEM'
    nbaTeams['Miami Heat'] = 'MIA'
    nbaTeams['Milwaukee Bucks'] = 'MIL'
    nbaTeams['Minnesota Timberwolves'] = 'MIN'
    nbaTeams['New Orleans Pelicans'] = 'NOP'
    nbaTeams['New York Knicks'] = 'NYK'
    nbaTeams['Oklahoma City Thunder'] = 'OKC'
    nbaTeams['Orlando Magic'] = 'ORL'
    nbaTeams['Philadelphia 76ers'] = 'PHI'
    nbaTeams['Phoenix Suns'] = 'PHX'
    nbaTeams['Portland Trail Blazers'] = 'POR'
    nbaTeams['Sacramento Kings'] = 'SAC'
    nbaTeams['San Antonio Spurs'] = 'SAS'
    nbaTeams['Toronto Raptors'] = 'TOR'
    nbaTeams['Utah Jazz'] = 'UTA'
    nbaTeams['Washington Wizards'] = 'WAS'
    nbaTeams_2 = {v: k for k, v in nbaTeams.items()}
    
    my_cursor = conn.cursor()
    my_cursor.execute('''Create view Jointed_Table as 
    select * from Shot_Selection natural join (select PlayerName, TeamName from Player_Bio) as short_bio;''')

    my_cursor.execute('''Create view Team_Shooting as
    select TeamName, sum(3FGA) as Total_3pt_attempt, sum(2FGA) as Total_2pt_attempt from Jointed_Table group by TeamName;''')

    
    sqlformula = "SELECT Total_3pt_attempt FROM Team_Shooting where TeamName = %s"
    my_cursor.execute(sqlformula, TeamName)
    my_result = my_cursor.fetchall()
    Three_ptA = int(my_result[0][0])
    
    sqlformula = '''select count(*)+1 from Team_Shooting 
    where Total_3pt_attempt > %s;'''

    my_cursor.execute(sqlformula, Three_ptA)
    my_result = my_cursor.fetchall()
    ranking = my_result[0][0]
    
    print("The team %s attempted %s three pointers during regular season last year, which ranked No.%d in the league." % (nbaTeams_2[TeamName], Three_ptA, ranking))
    
    my_cursor.execute("drop view Jointed_Table;")
    my_cursor.execute("drop view Team_Shooting")
    conn.close()

In [30]:
user_comments("James Harden", "Xiao Ming", 3, "I hate seeing him scoring 40 while is 2/16")

Successfully commented, your commentID is 3.


In [22]:
return_playerBio("James Harden")

     PlayerName TeamName  AvgPoints  AvgRebounds  AvgAssists  AvgSteals  AvgRating
0  James Harden      HOU       36.1          6.6         7.5        2.0        3.0


In [31]:
delete_ratings(3)

Successfully deleted, the average rating of James Harden has been adjusted.


In [24]:
return_playerBio("James Harden")

     PlayerName TeamName  AvgPoints  AvgRebounds  AvgAssists  AvgSteals  AvgRating
0  James Harden      HOU       36.1          6.6         7.5        2.0        0.0


In [25]:
query_stats_by_month("LeBron James", "Feb")

     PlayerName  Points  Rebounds  Assists  Steals  Game_Played
0  LeBron James  25.625     9.625   10.625   2.125            8


In [28]:
query_stats_by_opponent("LeBron James", "POR")

     PlayerName OpponentTeam   Points  Rebounds  Assists  Steals  Game_Played
0  LeBron James          POR  32.6667       9.0   7.3333  0.6667            3


In [116]:
query_all_results_groupby_opponent("Jeremy Lin")

    PlayerName OpponentTeam   Points  Rebounds  Assists  Steals  Game_Played
0   Jeremy Lin          MIN   7.5000    3.5000   4.5000  1.5000            2
1   Jeremy Lin          CHA   8.8000    1.6000   2.2000  0.8000            5
2   Jeremy Lin          BKN   8.0000    2.0000   1.6667  1.0000            3
3   Jeremy Lin          ORL   9.5000    2.0000   2.7500  0.0000            4
4   Jeremy Lin          CHI   8.0000    3.0000   3.0000  0.5000            4
5   Jeremy Lin          NYK  12.0000    3.8000   3.4000  0.4000            5
6   Jeremy Lin          OKC   5.5000    2.0000   0.5000  0.5000            2
7   Jeremy Lin          DET   8.2500    2.2500   3.2500  1.0000            4
8   Jeremy Lin          LAL   7.5000    4.0000   1.5000  0.0000            2
9   Jeremy Lin          CLE   6.7500    2.2500   3.2500  0.2500            4
10  Jeremy Lin          MIA   9.7500    2.2500   3.0000  1.0000            4
11  Jeremy Lin          NOP  14.0000    1.0000   3.0000  0.0000            1

In [56]:
find_best_clutch_FT_Shooter("TOR")

      PlayerName  Free_Throw_P TeamName  Minutes_Played
0  Fred VanVleet         100.0      TOR              77
1     Jeremy Lin         100.0      TOR              23


In [124]:
find_team_3PTA_ranking("GSW")

The team Golden State Warriors attempted 2791 three pointers during regular season last year, which ranked No.10 in the league.
